In [4]:
import pandas as pd

df = pd.read_csv("../data/SisFall_df_sc.csv")

print(df)

       user_ID trial_ID task_ID         gx         gy         gz     label   
0         SA01      R01     D01  -1.098633 -30.761719 -21.484375   walking  \
1         SA01      R01     D01 -10.864258 -46.752930  -3.173828   walking   
2         SA01      R01     D01  31.860352 -22.216797   8.056641   walking   
3         SA01      R01     D01   2.624512 -11.352539  29.052734   walking   
4         SA01      R01     D01   7.263184  15.869141  26.184082   walking   
...        ...      ...     ...        ...        ...        ...       ...   
331797    SE15      R05     D07  -1.281738   3.601074  -0.366211  standing   
331798    SE15      R05     D07  -1.586914   3.295898  -0.610352  standing   
331799    SE15      R05     D07  -1.586914   3.173828  -0.915527  standing   
331800    SE15      R05     D07  -1.525879   2.868652  -0.976562  standing   
331801    SE15      R05     D07  -1.281738   3.051758  -0.915527  standing   

        scaled_gx  scaled_gy  scaled_gz  
0        0.015592   0

In [17]:
import numpy as np
import pandas as pd
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pickle

# 데이터셋 준비
X = []  # 입력 데이터
y = []  # 출력 데이터

# 4개의 레코드씩 묶어서 처리
for i in range(0, len(df) - 3, 4):
    # gx, gy, gz 값 가져오기
    gx_values = df.loc[i:i+3, 'scaled_gx'].values
    gy_values = df.loc[i:i+3, 'scaled_gy'].values
    gz_values = df.loc[i:i+3, 'scaled_gz'].values

    # 레이블 값 가져오기
    labels = df.loc[i:i+3, 'label'].values

    # 서로 다른 레이블이 포함된 경우 해당 시퀀스는 분석에서 제외
    if len(set(labels)) == 1:
        # 시퀀스로 변환하여 입력 데이터에 추가
        X.append([gx_values, gy_values, gz_values])

        # 레이블 값 중복 제거하여 출력 데이터에 추가
        y.append(labels[0])

# 입력 데이터와 출력 데이터를 넘파이 배열로 변환
X = np.array(X)
y = np.array(y)

# 출력 데이터를 One-Hot 인코딩
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
onehot_encoder = OneHotEncoder(sparse_output=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y = onehot_encoder.fit_transform(integer_encoded)

# LSTM 모델 정의
model = keras.models.Sequential()
model.add(keras.layers.LSTM(128, input_shape=(3, 4)))  # 3개의 시퀀스, 각 시퀀스에 4개의 피처
model.add(keras.layers.Dense(5, activation='softmax'))  # 분류할 클래스 수에 맞게 조정

# 모델 컴파일 및 학습
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X, y, epochs=100, batch_size=32)

# 최상의 적합 모델의 가중치를 피클 파일로 저장
best_model_weights = history.model.get_weights()
pickle.dump(best_model_weights, open('../model/best_model_weights_cnn.pkl', 'wb'))

# 레이블 인코더의 클래스 정보를 저장
np.save('../model/label_encoder_classes_cnn.npy', label_encoder.classes_)

# 가중치를 로드하기 위해 모델 구성
loaded_model = keras.models.Sequential()
loaded_model.add(keras.layers.LSTM(128, input_shape=(3, 4)))  # 3개의 시퀀스, 각 시퀀스에 4개의 피처
loaded_model.add(keras.layers.Dense(5, activation='softmax'))  # 분류할 클래스 수에 맞게 조정

# 피클 파일에서 가중치 로드
loaded_model_weights = pickle.load(open('../model/best_model_weights_cnn.pkl', 'rb'))

# 모델에 로드된 가중치 설정
loaded_model.set_weights(loaded_model_weights)

# 모델 컴파일
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 저장
loaded_model.save('../model/loaded_model_cnn.h5')

# 예측할 데이터 불러오기
pred_df = pd.read_csv('../data/Arduino/fall_sc_cnn.csv')

# 입력 데이터 준비
X_pred = []  # 입력 데이터

# 4개의 레코드씩 묶어서 처리
for i in range(0, len(pred_df) - 3, 4):
    # gx, gy, gz 값 가져오기
    gx_values = pred_df.loc[i:i+3, 'scaled_gx'].values
    gy_values = pred_df.loc[i:i+3, 'scaled_gy'].values
    gz_values = pred_df.loc[i:i+3, 'scaled_gz'].values

    # 데이터 형상 조정
    gx_values = np.transpose(gx_values)
    gy_values = np.transpose(gy_values)
    gz_values = np.transpose(gz_values)

    # 시퀀스로 변환하여 입력 데이터에 추가
    X_pred.append([gx_values, gy_values, gz_values])

# 입력 데이터를 넘파이 배열로 변환
X_pred = np.array(X_pred)

# 모델 로드
try:
    best_model = keras.models.load_model('../model/loaded_model_cnn.h5')
except FileNotFoundError as e:
    print(f"모델 파일을 찾을 수 없습니다: {e}")
    exit()

# 모델을 사용하여 예측
predictions = best_model.predict(X_pred)

# 예측 결과 디코딩
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load('../model/label_encoder_classes_cnn.npy')
decoded_predictions = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

# 예측 결과를 데이터프레임에 추가
pred_df['predicted_label_cnn'] = np.repeat(decoded_predictions, 4)

# 예측 결과를 CSV 파일로 저장
pred_df.to_csv('../data/prediction_results_cnn.csv', index=False)


Epoch 1/100
2590/2590 [==============================] - 7s 2ms/step - loss: 1.4584 - accuracy: 0.3500
Epoch 2/100
2590/2590 [==============================] - 5s 2ms/step - loss: 1.2018 - accuracy: 0.4780
Epoch 3/100
2590/2590 [==============================] - 5s 2ms/step - loss: 1.1664 - accuracy: 0.4956
Epoch 4/100
2590/2590 [==============================] - 5s 2ms/step - loss: 1.1092 - accuracy: 0.5207
Epoch 5/100
2590/2590 [==============================] - 5s 2ms/step - loss: 1.0512 - accuracy: 0.5505
Epoch 6/100
2590/2590 [==============================] - 5s 2ms/step - loss: 1.0113 - accuracy: 0.5694
Epoch 7/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.9722 - accuracy: 0.5918
Epoch 8/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.9518 - accuracy: 0.6021
Epoch 9/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.9383 - accuracy: 0.6086
Epoch 10/100
2590/2590 [==============================] - 5s 2ms/step - l

FileNotFoundError: [Errno 2] No such file or directory: '../data/Arduino/fall_sc_cnn.csv'

In [18]:
# 예측할 데이터 불러오기
pred_df = pd.read_csv('../data/Arduino/fall_sc.csv')

# 입력 데이터 준비
X_pred = []  # 입력 데이터

# 4개의 레코드씩 묶어서 처리
for i in range(0, len(pred_df) - 3, 4):
    # gx, gy, gz 값 가져오기
    gx_values = pred_df.loc[i:i+3, 'scaled_gx'].values
    gy_values = pred_df.loc[i:i+3, 'scaled_gy'].values
    gz_values = pred_df.loc[i:i+3, 'scaled_gz'].values

    # 데이터 형상 조정
    gx_values = np.transpose(gx_values)
    gy_values = np.transpose(gy_values)
    gz_values = np.transpose(gz_values)

    # 시퀀스로 변환하여 입력 데이터에 추가
    X_pred.append([gx_values, gy_values, gz_values])

# 입력 데이터를 넘파이 배열로 변환
X_pred = np.array(X_pred)

# 모델 로드
try:
    best_model = keras.models.load_model('../model/loaded_model_cnn.h5')
except FileNotFoundError as e:
    print(f"모델 파일을 찾을 수 없습니다: {e}")
    exit()

# 모델을 사용하여 예측
predictions = best_model.predict(X_pred)

# 예측 결과 디코딩
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load('../model/label_encoder_classes_cnn.npy')
decoded_predictions = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

# 예측 결과를 데이터프레임에 추가
pred_df['predicted_label_cnn'] = np.repeat(decoded_predictions, 4)

# 예측 결과를 CSV 파일로 저장
pred_df.to_csv('../data/prediction_results_cnn.csv', index=False)


2/2 [==============================] - 1s 5ms/step


In [22]:
# 데이터셋 준비
X = []  # 입력 데이터
y = []  # 출력 데이터

# 4개의 레코드씩 묶어서 처리
for i in range(0, len(df) - 3, 4):
    # gx, gy, gz 값 가져오기
    gx_values = df.loc[i:i+3, 'scaled_gx'].values
    gy_values = df.loc[i:i+3, 'scaled_gy'].values
    gz_values = df.loc[i:i+3, 'scaled_gz'].values

    # 레이블 값 가져오기
    labels = df.loc[i:i+3, 'label'].values

    # 서로 다른 레이블이 포함된 경우 해당 시퀀스는 분석에서 제외
    if len(set(labels)) == 1:
        # 시퀀스로 변환하여 입력 데이터에 추가
        X.append([gx_values, gy_values, gz_values])

        # 레이블 값 중복 제거하여 출력 데이터에 추가
        y.append(labels[0])

# 입력 데이터와 출력 데이터를 넘파이 배열로 변환
X = np.array(X)
y = np.array(y)

# 출력 데이터를 One-Hot 인코딩
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
onehot_encoder = OneHotEncoder(sparse_output=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y = onehot_encoder.fit_transform(integer_encoded)

In [23]:
# LSTM 모델 정의
model = keras.models.Sequential()
model.add(keras.layers.LSTM(128, input_shape=(3, 4)))  # 3개의 시퀀스, 각 시퀀스에 4개의 피처
model.add(keras.layers.Dense(5, activation='softmax'))  # 분류할 클래스 수에 맞게 조정

# 모델 컴파일 및 학습
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X, y, epochs=100, batch_size=32)

Epoch 1/100
2590/2590 [==============================] - 7s 2ms/step - loss: 1.5233 - accuracy: 0.3102
Epoch 2/100
2590/2590 [==============================] - 5s 2ms/step - loss: 1.2342 - accuracy: 0.4656
Epoch 3/100
2590/2590 [==============================] - 5s 2ms/step - loss: 1.1547 - accuracy: 0.5015
Epoch 4/100
2590/2590 [==============================] - 5s 2ms/step - loss: 1.0542 - accuracy: 0.5506
Epoch 5/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.9962 - accuracy: 0.5817
Epoch 6/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.9684 - accuracy: 0.5953
Epoch 7/100
2590/2590 [==============================] - 6s 2ms/step - loss: 0.9537 - accuracy: 0.6011
Epoch 8/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.9451 - accuracy: 0.6066
Epoch 9/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.9372 - accuracy: 0.6092
Epoch 10/100
2590/2590 [==============================] - 5s 2ms/step - l

In [24]:
# 최상의 적합 모델의 가중치를 피클 파일로 저장
best_model_weights = history.model.get_weights()
pickle.dump(best_model_weights, open('../model/best_model_weights.pkl', 'wb'))

# 레이블 인코더의 클래스 정보를 저장
np.save('../model/label_encoder_classes.npy', label_encoder.classes_)

In [26]:
# 가중치를 로드하기 위해 모델 구성
loaded_model = keras.models.Sequential()
loaded_model.add(keras.layers.LSTM(128, input_shape=(3, 4)))  # 3개의 시퀀스, 각 시퀀스에 4개의 피처
loaded_model.add(keras.layers.Dense(5, activation='softmax'))  # 분류할 클래스 수에 맞게 조정

# 피클 파일에서 가중치 로드
loaded_model_weights = pickle.load(open('../model/best_model_weights.pkl', 'rb'))

# 모델에 로드된 가중치 설정
loaded_model.set_weights(loaded_model_weights)

# 모델 컴파일
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 저장
loaded_model.save('../model/loaded_model_cnn.h5')

# 예측할 데이터 불러오기
pred_df = pd.read_csv('../data/Arduino/fall_sc.csv')

# 입력 데이터 준비
X_pred = []  # 입력 데이터

# 4개의 레코드씩 묶어서 처리
for i in range(0, len(pred_df) - 3, 4):
    # gx, gy, gz 값 가져오기
    gx_values = pred_df.loc[i:i+3, 'scaled_gx'].values
    gy_values = pred_df.loc[i:i+3, 'scaled_gy'].values
    gz_values = pred_df.loc[i:i+3, 'scaled_gz'].values

    # 데이터 형상 조정
    gx_values = np.transpose(gx_values)
    gy_values = np.transpose(gy_values)
    gz_values = np.transpose(gz_values)

    # 시퀀스로 변환하여 입력 데이터에 추가
    X_pred.append([gx_values, gy_values, gz_values])

# 입력 데이터를 넘파이 배열로 변환
X_pred = np.array(X_pred)

# 예측
predictions = loaded_model.predict(X_pred)
predicted_labels = np.argmax(predictions, axis=1)
predicted_classes = label_encoder.inverse_transform(predicted_labels)

# 예측 결과 출력
print(predicted_classes)

2/2 [==============================] - 0s 0s/step
['fall' 'fall' 'stairs_walking' 'stairs_walking' 'stairs_walking' 'fall'
 'jogging' 'fall' 'stairs_walking' 'fall' 'jogging' 'stairs_walking'
 'stairs_walking' 'fall' 'fall' 'fall' 'stairs_walking' 'stairs_walking'
 'fall' 'stairs_walking' 'fall' 'fall' 'stairs_walking' 'fall' 'fall'
 'fall' 'fall' 'fall' 'stairs_walking' 'stairs_walking' 'stairs_walking'
 'fall' 'stairs_walking' 'fall' 'jogging' 'jogging' 'fall'
 'stairs_walking' 'stairs_walking' 'jogging' 'stairs_walking' 'fall'
 'fall' 'fall' 'fall' 'stairs_walking' 'fall' 'jogging' 'fall'
 'stairs_walking']
